## 작업 경로 확인 및 varinf의 Repository clone
- 처음 1회만 수행
---

In [ ]:
!pwd

In [ ]:
!git clone https://github.com/varinf/TransformersDataAugmentation.git

## Install the trec datastests (not L2 datasets)
- 경로를 src/utils까지 이동한 후 trec datasets install을 수행
---

In [ ]:
%cd /home/delabgpu/Tae_StudyCode/TransformersDataAugmentation/src/utils/

In [ ]:
!bash download_and_prepare_datasets.sh

## Dependencies (jupyter ver.)
- CMD를 기본으로 설치하되, 설치하는 방법은 README를 참고.
- import 후 버전확인만 수행해볼 것.
- 이것은 jupyter에서 실행하는 방법을 기록함(실행할 필요 없음)

    * pytorch 1.5
    * fairseq 9.0
    * transformers 2.9
---

In [ ]:
pip install torch==1.5.0 torchvision==0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch

print(torch.__version__)

In [ ]:
pip install fairseq==0.9

In [ ]:
import fairseq

print(fairseq.__version__)

In [ ]:
pip install transformers==2.9.0

In [ ]:
import transformers

print(transformers.__version__)

## Prepare the L2-Datasets(or your datasets) and match data format - in README
---

In [ ]:
import pandas as pd

print(pd.__version__)

### Labeling and converting file format
- 맨 처음에만 수행할 것.

In [ ]:
''' 1. Make label -> Description '''

col=['Description']
list1 = []

for i in range(len(file)):
    list1.append('Description')
    
df = pd.DataFrame(list1, columns = col)
print(len(df))

file = pd.read_csv('Datasets L2+L3+ETGEN.txt', delimiter = '\t')
print(len(file))

result = pd.concat([df,file],axis=1, join='inner')


''' 2. txt -> tsv 변환하여 저장 '''
result.to_csv('L2_labeling_file.tsv', index=False, header=None, sep="\t")


### L2_labeling_file.tsv 를 train dev test 분할
- 맨 처음 한 번만 수행

In [ ]:
!pwd

%cd /home/delabgpu/Tae_StudyCode/

In [ ]:

result = pd.read_csv('L2_labeling_file.tsv', delimiter = '\t')

total_train = result[:264234]

total_dev = result[264234:528468]

total_test = result[528468:]


%cd TransformersDataAugmentation/src/utils/datasets/L2

total_train.to_csv('train.tsv', index=False, header=None, sep="\t")

total_dev.to_csv('dev.tsv', index=False, header=None, sep="\t")

total_test.to_csv('test.tsv', index=False, header=None, sep="\t")

### L2_labeling_file.tsv를 exp_{i}_10 폴더에 n등분 분할하여 배치
- 전체를 n등분
- 일부만 추출하여 n등분 (아래 코드는 이 방식을 사용)

In [ ]:
!pwd

In [ ]:
%cd /home/delabgpu/Tae_StudyCode/TransformersDataAugmentation/src/utils/datasets/L2

## train.tsv
- 0 ~ (3523 * 15) = 0~52831   완료
- 140,894 ~ (+ 3523 * 15)번째 = 140,894 ~ 193,725 완료
- 281,787 ~ (+ 52831번째) = 281,787 ~ 334,618 완료
- 422,680 ~ 475,511 완료
- 563,573 ~ 616,404 완료

In [ ]:
""" [ 첫 실행 시 < 전체에서 1/5만큼 > exp_j_10 / train.tsv 만들기 ] """

subset = []
last_num=0

for j in range(15):    
    for i in range(3523):
        subset = result[(j*i)+1:((j*i)+3523)]
        last_num = ((j*i)+3523)
    %cd exp_{j}_10
    subset.to_csv('train.tsv', index=False, header=None, sep="\t")
    %cd ../
    print("#########  [ ", j, "번째 성공 ]  ######")
    print("마지막숫자 : ", last_num)

''' 14번째 성공의 마지막 숫자 +1부터 dev만들기 '''

In [ ]:
""" [ 두 번째 이후부터 실행 시 < 전체에서 1/5 다음부터 1/5만큼 > exp_j_10 / train.tsv 만들기 ] """

result = result[140894+1:]
last_num=0

for j in range(15):    
    for i in range(3523):
        subset = result[(j*i)+1:((j*i)+3523)]
        last_num = ((j*i)+3523)
    %cd exp_{j}_10
    subset.to_csv('train.tsv', index=False, header=None, sep="\t")
    %cd ../
    print("#########  [ ", j, "번째 성공 ]  ######")
    print("마지막숫자 : ", last_num)

''' 14번째 성공의 마지막 숫자 +1부터 dev만들기 '''

## dev.tsv
- 52831 ~ (52831 + (3523 * 15)) = 52,831~105,662
- 193,725 ~ 246,556 완료
- 334,618 ~ 387,449 완료
- 475,511 ~ 528342 완료
- 616,404 ~ 669,235 완료

In [ ]:
""" [ < 전체에서 1/5만큼 > exp_j_10 / dev.tsv 만들기 ] """

result = result[last_num+1:]
subset = []

for j in range(15):    
    for i in range(3523):
        subset = result[(j*i)+1:((j*i)+3523)]
        last_num = ((j*i)+3523)
    %cd exp_{j}_10
    subset.to_csv('dev.tsv', index=False, header=None, sep="\t")
    %cd ../
    print("#########  [ ", j, "번째 성공 ]  ######")
    print("마지막숫자 : ", last_num)


In [ ]:
!pwd

In [ ]:
%cd /home/delabgpu/Tae_StudyCode

## test.tsv
- 105,662 ~ (105,662 + 35231) = 105,662 ~ 140,893
- 246,556 ~ 281,787 완료
- 387,449 ~ 422,680 완료
- 528,342 ~ 563,573 완료
- 669,235 ~ 704,466 완료

In [ ]:
''' 각 exp에 test.tsv 넣어줌 '''

result = result[last_num+1:]
test_result = result[:35231]   # 나중에 할 땐 범위 바꿔줘야함

%cd TransformersDataAugmentation/src/utils/datasets/L2

for j in range(15):        
    %cd exp_{j}_10
    test_result.to_csv('test.tsv', index=False, header=None, sep="\t")
    %cd ../
    print("#########  [ ", j, "번째 성공 ]  ######")

## Augmentation 수행 전, 경로 수정
1. /src/scripts/bert_L2_lower.sh 파일을 열어서 경로 수정 (아래 주석과 README파일 참고)
2. Datasets 폴더에 "L2"폴더를 생성한 후, exp_{0~14}_10 폴더를 만듦 
---

In [ ]:
'''
SRC=/home/delabgpu/Tae_StudyCode/TransformersDataAugmentation/src
CACHE=/home/delabgpu/Tae_StudyCode/TransformersDataAugmentation/CACHE
TASK=trec

for NUMEXAMPLES in 10;
do
    for i in {0..14};
        do
        RAWDATADIR=/home/delabgpu/Tae_StudyCode/TransformersDataAugmentation/src/utils/datasets/L2/exp_${i}_${NUMEXAMPLES}

       # Baseline classifier
        python $SRC/bert_aug/bert_classifier.py --task $TASK  --data_dir $RAWDATADIR --seed ${i} --learning_rate $BERTLR --cache $CACHE > $RAWDATADIR/bert_baseline.log

'''

In [ ]:
!pwd

In [ ]:
%cd /home/delabgpu/Tae_StudyCode/TransformersDataAugmentation/src/scripts

## Augmentation 수행
---

In [ ]:
!bash bert_L2_lower.sh